In [26]:
import os
import numpy as np
import pandas as pd
from sodapy import Socrata
import sqlalchemy

# Connect Postgres

In [27]:
# Get environment variables from .env & .env.secret files
pg_db = os.getenv('POSTGRES_DB')
pg_hostname = os.getenv('POSTGRES_HOSTNAME')
pg_port = os.getenv('POSTGRES_HOST_PORT')
pg_password = os.getenv('POSTGRES_PASSWORD')
pg_user = os.getenv('POSTGRES_USER')

# Build connection string
conn_str = 'postgresql://{}:{}@{}:{}/{}'.format(pg_user, pg_password, pg_hostname, pg_port, pg_db)

# Uncomment to debug postres connection issues:
# print('Connecting to: {}'.format(conn_str))

# create an `Engine` using the default psycopg2 adapter which uses the Python DBABI spec v2.
engine = sqlalchemy.create_engine(conn_str)

# The engine will ask the connection pool for a connection when the `connect()` or `execute()` methods are called.
conn = engine.connect()

# Violation Codes

## Load The Data

In [28]:
# Check that the violation_codes postgres table is fully populated.
try:
    result = conn.execute('SELECT count(*) from violation_codes;')
    for row in result:
        if row[0] < 934: # 934 records in .csv
            raise RuntimeWarning('WARNING: violation_codes table not populated correctly!')
        print(row)
except sqlalchemy.exc.SQLAlchemyError as e:
    print('Error fetching data from database! Run `make db-setup` and try again.\n')
    print(e)

(934,)


In [29]:
# Load the violation codes dataframe into pandas.
violation_codes_df = pd.read_sql(sql='SELECT * from violation_codes;', con=conn)

## Basic analysis

In [30]:
violation_codes_df.shape

(934, 14)

In [31]:
violation_codes_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 934 entries, 0 to 933
Data columns (total 14 columns):
violation_id      928 non-null object
org_id            925 non-null object
violation_type    925 non-null object
violation_desc    925 non-null object
violation_text    912 non-null object
remedial_text     788 non-null object
table_name        788 non-null object
expired_flag      788 non-null object
date_expired      788 non-null object
created_by        788 non-null object
date_created      651 non-null object
modified_by       651 non-null object
date_modified     651 non-null object
fee_setup_id      651 non-null object
dtypes: object(14)
memory usage: 102.2+ KB


In [32]:
violation_codes_df.describe()

,violation_id,org_id,violation_type,violation_desc,violation_text,remedial_text,table_name,expired_flag,date_expired,created_by,date_created,modified_by,date_modified,fee_setup_id
count,928,925,925,925,912,788,788,788,788,788,651,651,651,651
unique,810,3,789,773,514,211,12,3,3,6,42,4,23,1
top,-,1,NULL,N,NULL,NULL,NULL,N,NULL,2,00:00.0,NULL,NULL,NULL
freq,119,788,137,137,137,443,651,650,786,405,397,628,629,651


In [33]:
def vc_drop_values():
    return violation_codes_df.query('violation_type == "NULL" and violation_desc == "N" and violation_text == "NULL"').index

mask = vc_drop_values()
violation_codes_df.drop(mask, inplace=True)
violation_codes_df.shape

(797, 14)

In [34]:
def isnumeric(x):
    try:
        int(x)
        return True
    except:
        return False
    
numeric = violation_codes_df['violation_id'].apply(isnumeric)
violation_codes_df['violation_text'][numeric].str.startswith('Section').value_counts()

True     429
False    346
Name: violation_text, dtype: int64

In [35]:
def violation_text_startswith(s):
    if s == None:
        return 'None'
    if s.find('T.C.A') > -1:
        return 'TCA'
    elif s.find('M.C.L') > -1 or s.find('MCL') > -1:
        return 'MCL'
    else:
        return 'Other'
    
violation_codes_df['violation_text_type'] = violation_codes_df['violation_text'].apply(violation_text_startswith)
violation_codes_df.violation_text_type.value_counts()

Other    597
MCL       98
TCA       80
None      22
Name: violation_text_type, dtype: int64

In [36]:
violation_codes_df['MCL'] = violation_codes_df['violation_text'][numeric].str.startswith('M.C.L')

In [37]:
vc_dict = violation_codes_df.violation_type.to_dict()
violation_codes_df.fillna(value=vc_dict)

,violation_id,org_id,violation_type,violation_desc,violation_text,remedial_text,table_name,expired_flag,date_expired,created_by,date_created,modified_by,date_modified,fee_setup_id,violation_text_type,MCL
0,1,1,BANNERS,Banners,Section 17.32.060 & 17.32.070 - Banners: Bann...,CAAZ_BANNERS,NULL,N,NULL,1,03:52.6,NULL,NULL,NULL,Other,False
1,2,1,BGMATOWNER,Proper Maintenance Req - Owner,Section 16.16.030 (B) - Proper maintenance req...,CAAH_BG_MATOWNER,NULL,N,NULL,1,03:52.6,NULL,NULL,NULL,Other,False
2,4,1,BLDGMAINT,Proper Maintenance Req,Section 16.16.030 (A) - Proper maintenance req...,CAAH_BLDG_MAINT,NULL,N,NULL,1,03:52.6,NULL,NULL,NULL,Other,False
3,5,1,BLDGPERMIT,Building Permit Required,Section 16.28.010 - Building Permit Required: ...,CAAB_BLDG_PERMIT,NULL,N,NULL,1,03:52.6,NULL,NULL,NULL,Other,False
4,6,1,BLDGSCOPE,Scope of Building Code,Section 16.16.020 (B) - The provisions of this...,CAAH_BLDG_SCOPE,NULL,N,NULL,1,03:52.6,NULL,NULL,NULL,Other,False
5,7,1,BOARDINGRQ,Boarding Requirements,Section 16.24.340 (U) (2) – Boarding of Vacant...,CAAH_BOARDING_REQU,NULL,N,NULL,1,03:52.6,NULL,NULL,NULL,Other,False
6,8,1,BOARDVAC,Boarding of Vacant Buildings,Section 16.24.340 (U) (1) – Boarding of Vacant...,CAAH_BOARDING_VABLDG,NULL,N,NULL,1,03:52.6,NULL,NULL,NULL,Other,False
7,9,1,CARROW,Vehicle In The Right-Of-Way,Section 12.08.210 - Abandoned Vehicles: Abando...,CAMCL_SEW_CONCT,NULL,N,NULL,1,03:52.6,NULL,NULL,NULL,Other,False
8,10,1,CERTCOMP,Certificate of Compliance Required,Section 17.40.580 - Certificate of Compliance:...,CAAZ_CERT_COMP,NULL,N,NULL,1,03:52.6,NULL,NULL,NULL,Other,False
9,11,1,CERTOCC,Certificate of Occupancy Required,Section 16.24.130 – Certificate of Occupancy R...,CAAH_CERT_OCC,NULL,N,NULL,1,03:52.6,NULL,NULL,NULL,Other,False


# Property Standards Violations Data

## Helpers

In [38]:
def get_property_standards_violations_update(save=False):
    """
        Socrata client pulls updated data from https://data.nashville.gov
        
        Property Standards Violations data docs: https://dev.socrata.com/foundry/data.nashville.gov/tcjq-k4c5
        
        Socrata client docs: https://dev.socrata.com/foundry/data.nashville.gov/tcjq-k4c5
        
        Dataframe will be saved to .csv file if save=True is passed as argument.
    """
    
    socrata_app_token = os.getenv('SOCRATA_APP_TOKEN')
    client = Socrata("data.nashville.gov", socrata_app_token)
    results = client.get("tcjq-k4c5", limit=100000) # dataset has > 51,000 records
    psv_df = pd.DataFrame.from_records(results)
    
    # clean df
    for column in psv_df.columns:
        if column.startswith(':@'):
            psv_df.drop(column, axis=1, inplace=True)
    
    # optionally save to .csv
    if save == True:
        save_update_to_csv(psv_df, 'property-standards-violations')
    
    return psv_df

def save_update_to_csv(df, name):
    filename = './data/{}-update.csv'.format(name)
    df.to_csv(filename, index=False)

Optionally get an update of the data (the shape can be diffed with the current df):

In [39]:
# get_property_standards_violations_update(save=True).shape

# Load The Data

In [40]:
# Check that the property_standards_violations postgres table is fully populated.
try:
    result = conn.execute('SELECT count(*) from property_standards_violations;')
    for row in result:
        if row[0] < 51696: # records in original .csv
            raise RuntimeWarning('WARNING: property_standards_violations table not populated correctly!')
        print(row)
except sqlalchemy.exc.SQLAlchemyError as e:
    print('Error fetching data from database! Run `make db-setup` and try again.\n')
    print(e)

(51696,)


In [41]:
# Load the property standards violations dataframe into pandas.
psv_df = pd.read_sql(sql='SELECT * from property_standards_violations;', con=conn)

## Basic analysis

In [115]:
psv_df.shape

(51696, 21)

In [112]:
psv_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51696 entries, 0 to 51695
Data columns (total 21 columns):
city                       51077 non-null object
complaint_source           51602 non-null object
council_district           51159 non-null object
date_received              51696 non-null object
last_activity              51332 non-null object
last_activity_date         51332 non-null object
last_activity_result       51320 non-null object
mapped_location            50626 non-null object
mapped_location_address    51317 non-null object
mapped_location_city       51077 non-null object
mapped_location_state      51127 non-null object
mapped_location_zip        51272 non-null object
property_address           51317 non-null object
property_apn               51260 non-null object
property_owner             51696 non-null object
reported_problem           50990 non-null object
request                    51696 non-null object
state                      51127 non-null object
status   

In [121]:
psv_df['council_district']

0         3
1        29
2        11
3        11
4        26
5        17
6        16
7         9
8        20
9        11
10       13
11       11
12       27
13       29
14       16
15       22
16       18
17       21
18       16
19       11
20       16
21       16
22       16
23       11
24       30
25       21
26       21
27       13
28       29
29       13
         ..
51666    17
51667    13
51668    11
51669     6
51670     4
51671    31
51672    28
51673    14
51674    16
51675    12
51676    11
51677    12
51678    12
51679    35
51680    30
51681     8
51682    16
51683    11
51684    22
51685     5
51686    24
51687    24
51688    26
51689    15
51690    15
51691    28
51692    15
51693    24
51694     6
51695    11
Name: council_district, Length: 51696, dtype: object